
# Insurance Pricing, Survival Analysis & XGBoost Tuning (Optuna)

This notebook demonstrates an end‑to‑end workflow on the classical **`insurance.csv`** dataset:

1. **Data loading & basic exploration**
2. **Pricing models**
   - Generalized Linear Model (**GLM**) with a **Tweedie** family
   - **Generalized Additive Model (GAM)** for nonlinear pricing effects
3. **Survival analysis**
   - **Kaplan–Meier** curves
   - **Cox proportional hazards** model
4. **XGBoost regression** for claim cost with **hyperparameter tuning using Optuna**

> ⚠️ **Note**: Adjust column names/paths if your file differs from the classic `insurance.csv` (age, sex, bmi, children, smoker, region, charges).


In [ ]:

# Core libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# For reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Setup complete.")


## 1. Load and Explore the Data

In [ ]:

# Path to the dataset (put insurance.csv in the same folder as this notebook)
DATA_PATH = "insurance.csv"  # change this if needed

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()


In [ ]:

df.info()


In [ ]:

df.describe(include="all")


In [ ]:

# Quick visualization: distribution of charges and some relationships
plt.figure(figsize=(6,4))
sns.histplot(df["charges"], bins=40)
plt.title("Distribution of Charges")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
sns.scatterplot(data=df, x="age", y="charges", hue="smoker")
plt.title("Charges vs Age by Smoker Status")
plt.tight_layout()
plt.show()



## 2. Pricing Models

We will build pricing models with:

- **GLM with Tweedie family** (compound Poisson–Gamma, useful for claim severity/frequency)
- **GAM (pyGAM)** to capture nonlinear effects.


In [ ]:

# Target: charges (claim cost / premium)
y = df["charges"]

# Features: all except charges
X = df.drop(columns=["charges"])

# One‑hot encode categorical features for the GLM and tree-based models
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)

X_encoded.head()



### 2.1 GLM with Tweedie Family (using `statsmodels`)

We model charges using a Tweedie distribution with a **log link**, suitable for positive, right‑skewed data.


In [ ]:

import statsmodels.api as sm
from statsmodels.genmod.families import Tweedie

X_glm = sm.add_constant(X_encoded)  # add intercept
y_glm = y

glm_model = sm.GLM(
    y_glm,
    X_glm,
    family=sm.families.Tweedie(
        var_power=1.5,             # 1 < p < 2 → compound Poisson–Gamma
        link=sm.families.links.log()
    )
)

glm_res = glm_model.fit()
print(glm_res.summary())


In [ ]:

# Predicted vs actual from GLM
y_pred_glm = glm_res.predict(X_glm)

rmse_glm = mean_squared_error(y, y_pred_glm, squared=False)
r2_glm = r2_score(y, y_pred_glm)
print(f"GLM Tweedie RMSE: {rmse_glm:,.2f}")
print(f"GLM Tweedie R²:   {r2_glm:,.3f}")

plt.figure(figsize=(6,6))
plt.scatter(y, y_pred_glm, alpha=0.4)
plt.plot([y.min(), y.max()], [y.min(), y.max()], "r--")
plt.xlabel("Actual charges")
plt.ylabel("Predicted charges (GLM Tweedie)")
plt.title("Actual vs Predicted Charges (GLM Tweedie)")
plt.tight_layout()
plt.show()



### 2.2 Generalized Additive Model (GAM) using `pyGAM`

We fit a GAM to allow smooth nonlinear effects for all predictors.


In [ ]:

# Install pyGAM if not already installed (uncomment the next line when running locally)
# %pip install pygam

from pygam import LinearGAM, s

# We'll reuse X_encoded (all numeric after one‑hot encoding)
X_gam = X_encoded.copy()
y_gam = y.values

# Build a GAM with a spline term s(i) for each feature
terms = sum([s(i) for i in range(X_gam.shape[1])])
gam = LinearGAM(terms)

gam.gridsearch(X_gam.values, y_gam)
print(gam.summary())


In [ ]:

# Evaluate GAM
y_pred_gam = gam.predict(X_gam.values)
rmse_gam = mean_squared_error(y_gam, y_pred_gam, squared=False)
r2_gam = r2_score(y_gam, y_pred_gam)
print(f"GAM RMSE: {rmse_gam:,.2f}")
print(f"GAM R²:   {r2_gam:,.3f}")

plt.figure(figsize=(6,6))
plt.scatter(y_gam, y_pred_gam, alpha=0.4)
plt.plot([y.min(), y.max()], [y.min(), y.max()], "r--")
plt.xlabel("Actual charges")
plt.ylabel("Predicted charges (GAM)")
plt.title("Actual vs Predicted Charges (GAM)")
plt.tight_layout()
plt.show()


In [ ]:

# Plot partial dependence for a few key features
feature_names = X_gam.columns.tolist()

for idx in [feature_names.index(col) for col in ["age", "bmi"] if col in feature_names]:
    XX = gam.generate_X_grid(term=idx)
    plt.figure(figsize=(6,4))
    plt.plot(XX[:, idx], gam.partial_dependence(term=idx, X=XX))
    plt.plot(XX[:, idx], gam.partial_dependence(term=idx, X=XX, width=0.95)[1], linestyle="--")
    plt.title(f"Partial dependence for {feature_names[idx]}")
    plt.xlabel(feature_names[idx])
    plt.ylabel("Effect on charges")
    plt.tight_layout()
    plt.show()



## 3. Survival Analysis (Kaplan–Meier & Cox)

The original `insurance.csv` does **not** contain time‑to‑event and censoring information.
To illustrate survival models, we will **simulate** a survival dataset based on age and smoker status.

In a real life insurance application, you would replace this simulated part with your true:
- `duration` (time to death / lapse / claim)
- `event` indicator (1 if event occurred, 0 if censored)


In [ ]:

# Install lifelines if not already installed (uncomment when running locally)
# %pip install lifelines

from lifelines import KaplanMeierFitter, CoxPHFitter

surv_df = df.copy()

# Simulate survival times (purely illustrative)
# Hazard increases with age and smoker status
baseline_hazard = 0.02
age_effect = 0.04
smoker_effect = 0.8

lambda_rate = baseline_hazard * np.exp(
    age_effect * (surv_df["age"] - surv_df["age"].mean()) / surv_df["age"].std()
    + smoker_effect * (surv_df["smoker"] == "yes").astype(int)
)

surv_df["time"] = np.random.exponential(1 / lambda_rate)
surv_df["event"] = np.random.binomial(1, 0.7, size=len(surv_df))  # 70% observed events

surv_df[["age", "smoker", "time", "event"]].head()


### 3.1 Kaplan–Meier Curves

In [ ]:

kmf = KaplanMeierFitter()

plt.figure(figsize=(6,4))
kmf.fit(durations=surv_df["time"], event_observed=surv_df["event"], label="All")
kmf.plot_survival_function()
plt.title("Kaplan–Meier Survival Curve (All)")
plt.xlabel("Time")
plt.ylabel("Survival probability")
plt.tight_layout()
plt.show()

# Separate curves for smokers vs non‑smokers
plt.figure(figsize=(6,4))
for group_value, group_df in surv_df.groupby("smoker"):
    kmf.fit(group_df["time"], group_df["event"], label=f"smoker={group_value}")
    kmf.plot_survival_function()

plt.title("Kaplan–Meier by Smoker Status")
plt.xlabel("Time")
plt.ylabel("Survival probability")
plt.tight_layout()
plt.show()


### 3.2 Cox Proportional Hazards Model

In [ ]:

# Build a Cox model dataframe: duration, event + covariates
cox_df = surv_df[["time", "event", "age", "bmi", "children"]].copy()

# Add one‑hot encoded categorical covariates
cox_covars = pd.get_dummies(
    surv_df[["sex", "smoker", "region"]],
    drop_first=True
)
cox_df = pd.concat([cox_df, cox_covars], axis=1)

cph = CoxPHFitter()
cph.fit(cox_df, duration_col="time", event_col="event")
cph.print_summary()


In [ ]:

# Plot Cox model coefficients
cph.plot()
plt.title("Cox Model Coefficients")
plt.tight_layout()
plt.show()



## 4. XGBoost Regression with Optuna Hyperparameter Tuning

We use XGBoost to model `charges` and Optuna to search for good hyperparameters.


In [ ]:

# Install xgboost and optuna if needed (uncomment when running locally)
# %pip install xgboost optuna

import xgboost as xgb
import optuna

# Reuse the encoded features
X_reg = X_encoded.copy()
y_reg = y.copy()

X_train, X_valid, y_train, y_valid = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "auto",
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 600),
    }

    model = xgb.XGBRegressor(**params, random_state=RANDOM_STATE)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, preds, squared=False)
    return rmse


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, show_progress_bar=False)

print("Best RMSE:", study.best_value)
print("Best hyperparameters:")
study.best_params


In [ ]:

best_params = study.best_params

best_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    eval_metric="rmse",
    tree_method="auto",
    random_state=RANDOM_STATE,
    **best_params
)

best_model.fit(X_train, y_train)
y_pred_xgb = best_model.predict(X_valid)

rmse_xgb = mean_squared_error(y_valid, y_pred_xgb, squared=False)
r2_xgb = r2_score(y_valid, y_pred_xgb)

print(f"XGBoost (tuned) RMSE: {rmse_xgb:,.2f}")
print(f"XGBoost (tuned) R²:   {r2_xgb:,.3f}")


In [ ]:

plt.figure(figsize=(6,6))
plt.scatter(y_valid, y_pred_xgb, alpha=0.4)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], "r--")
plt.xlabel("Actual charges (validation)")
plt.ylabel("Predicted charges (XGBoost tuned)")
plt.title("Actual vs Predicted Charges (XGBoost + Optuna)")
plt.tight_layout()
plt.show()



## 5. Summary

In this notebook, we:

- Built a **GLM Tweedie** pricing model for insurance charges.
- Fitted a **GAM** to capture nonlinear relationships.
- Illustrated **survival analysis** with **Kaplan–Meier** and **Cox PH** models (using simulated durations).
- Trained an **XGBoost** regressor and tuned hyperparameters with **Optuna**.

You can adapt this structure to your own life insurance or non‑life datasets by:
- Replacing `charges` with your claim or premium variable.
- Providing real survival times (`duration`) and event indicators (`event`) for the survival analysis section.
- Adding exposure offsets and frequency/severity decomposition as needed.
